Fuction to create the file if the newest dataframe exists

In [1]:
import pandas as pd     # *****************************************************

def create_newest_df(path, colums):
    if not path.exists():
        file = pd.DataFrame(columns=colums)
        file.to_csv(path, index=False)
        print("File created")
        return file
    else:
        return pd.read_csv(path)

Reading the existent number of movies that are presented in the files

In [2]:
from pathlib import Path   #***************************************************

letterboxd_data = Path("data/diary.csv")
updated_letterboxd_data = Path("data/newer_diary.csv")

df = pd.read_csv(letterboxd_data, sep=",")
newest_df = create_newest_df(updated_letterboxd_data, df.columns)

print(len(df), len(newest_df))

118 118


Show the dataframe from letterboxd

In [3]:
print(df.to_string())

           Date                                     Name  Year          Letterboxd URI  Rating Rewatch  Tags Watched Date
0    2023-12-30                                    Venom  2018  https://boxd.it/5r9pCZ     3.0     NaN   NaN   2020-09-18
1    2023-12-31                                  Get Out  2017  https://boxd.it/5rMbdz     4.0     NaN   NaN   2020-09-23
2    2025-01-08                          The New Mutants  2020  https://boxd.it/8mRaeb     1.5     NaN   NaN   2020-11-12
3    2023-12-30                        Wonder Woman 1984  2020  https://boxd.it/5r9y0F     3.5     NaN   NaN   2020-12-28
4    2023-12-30                         The Longest Yard  2005  https://boxd.it/5r9w2f     3.5     NaN   NaN   2021-03-17
5    2023-12-30             Zack Snyder's Justice League  2021  https://boxd.it/5r9Dz1     4.5     NaN   NaN   2021-03-18
6    2023-12-30                               The Father  2020  https://boxd.it/5r9tQV     4.0     NaN   NaN   2021-04-28
7    2023-12-30         

_Picking up all movie information from OMDB_

EXAMPLE OF API OUTPUT

Title => Inception
Year => 2010
Rated => PG-13
Released => 16 Jul 2010
Runtime => 148 min
Genre => Action, Adventure, Sci-Fi
Director => Christopher Nolan
Writer => Christopher Nolan
Actors => Leonardo DiCaprio, Joseph Gordon-Levitt, Elliot Page

Plot => A thief who steals corporate secrets through the use of dream-sharing technology is given the inverse task of planting an idea into the mind of a C.E.O., but his tragic past may doom the project and his team to disaster.

Language => English, Japanese, French
Country => United States, United Kingdom
Awards => Won 4 Oscars. 159 wins & 220 nominations total
Poster => https://m.media-amazon.com/images/M/MV5BMjAxMzY3NjcxNF5BMl5BanBnXkFtZTcwNTI5OTM0Mw@@._V1_SX300.jpg

Ratings:
 - Internet Movie Database: 8.8/10
 - Rotten Tomatoes: 87%
 - Metacritic: 74/100

Metascore => 74
imdbRating => 8.8
imdbVotes => 2,626,859
imdbID => tt1375666
Type => movie
DVD => N/A

In [16]:
# Filtra filmes que não estão em newest_df #*************************************************
df_to_process = df[~df['Name'].isin(newest_df['Name'])]
print(df_to_process.to_string())


           Date                              Name  Year          Letterboxd URI  Rating Rewatch  Tags Watched Date
109  2025-03-10                 Kill Bill: Vol. 2  2004  https://boxd.it/957V9l     4.0     NaN   NaN   2025-03-09
110  2025-03-15  Captain America: Brave New World  2025  https://boxd.it/97Vxqv     2.5     NaN   NaN   2025-03-14
111  2025-04-11                         Mickey 17  2025  https://boxd.it/9nn7n7     3.0     NaN   NaN   2025-04-11
112  2025-04-14                          Longlegs  2024  https://boxd.it/9oPa9R     2.5     NaN   NaN   2025-04-13
113  2025-05-02                      A Dog's Will  1999  https://boxd.it/9zL7E9     4.5     NaN   NaN   2025-05-01
114  2025-05-03                        The Monkey  2025  https://boxd.it/9AhmZ1     2.0     NaN   NaN   2025-05-02
115  2025-05-04                    A Dog's Will 2  2024  https://boxd.it/9APC6J     2.5     NaN   NaN   2025-05-03
116  2025-05-10                    The Accountant  2016  https://boxd.it/9Eaeyt 

In [17]:
import traceback  #*****************************************
import requests

BASE_URL = "http://www.omdbapi.com/"

if df_to_process.empty:
    print("There are no new movies to process. Exiting.")

else:
    for i, row in df_to_process.iterrows():
        params = {"apikey": "77055a01", "t": row["Name"], "y": row["Year"]}
        try:
            response = requests.get(BASE_URL, params=params)
            data = response.json()

            if "Title" in data and row["Name"].lower() == data["Title"].lower():
                print(f"Filme {i}: {row['Name']} encontrado")
            else:
                print(f"\n ******* Título não corresponde ou não encontrado: {row['Name']} != {data.get('Title', 'N/A')} ******* \n")
        except Exception as e:
            print(f"Erro no índice {i}: {e}")
            print("Traceback:")
            traceback.print_exc()


Filme 109: Kill Bill: Vol. 2 encontrado
Filme 110: Captain America: Brave New World encontrado
Filme 111: Mickey 17 encontrado
Filme 112: Longlegs encontrado
Filme 113: A Dog's Will encontrado
Filme 114: The Monkey encontrado
Filme 115: A Dog's Will 2 encontrado
Filme 116: The Accountant encontrado
Filme 117: Oeste Outra Vez encontrado


Making the necessary changes in the incompatible movie names

In [15]:
df.loc[91, "Name"] = "Planes, Trains & Automobiles"     #**********************************************
df.loc[83, "Name"] = "Horizon: An American Saga - Chapter 1"
df.loc[50, "Name"] = "Godzilla x Kong: The New Empire"
df.loc[37, "Name"] = "Gone in 60 Seconds"
df.loc[28, "Name"] = "El Camino"
df.loc[100, "Name"] = "Elite Squad 2: The Enemy Within"
df.loc[46, "Name"] = "Dune: Part One"
df.loc[113, "Name"] = "A Dog's Will"
df.loc[115, "Name"] = "A Dog's Will 2"
df.loc[117, "Name"] = "Oeste Outra Vez"

df.to_csv(letterboxd_data, index=False)

In [18]:
import pandas as pd   #**********************************

if df_to_process.empty:
    print("There are no new movies to process. Exiting.")

else:
    for i, row in df_to_process.iterrows():
        params = {"apikey": "77055a01", "t": row["Name"], "y": row["Year"]}
        try:
            response = requests.get(BASE_URL, params=params)
            data = response.json()

            print(f"Processando filme {i}: {row['Name']}")

            # Verificando se o título retornado pela API corresponde
            if "Title" in data and row["Name"].lower() == data["Title"].lower():
                new_info = {
                    "Name": row.get("Name", ""),
                    "Year": row.get("Year", ""),
                    "Letterboxd URI": row.get("Letterboxd URI", ""),
                    "Rating": row.get("Rating", ""),
                    "Watched Date": row.get("Watched Date", ""),
                    "Runtime": data.get("Runtime", ""),
                    "Genre": data.get("Genre", ""),
                    "Director": data.get("Director", ""),
                    "Writer": data.get("Writer", ""),
                    "Actors": data.get("Actors", ""),
                    "Language": data.get("Language", ""),
                    "Country": data.get("Country", ""),
                    "Poster": data.get("Poster", "")
                }
                # Adiciona a nova linha ao DataFrame de destino
                newest_df = pd.concat([newest_df, pd.DataFrame([new_info])], ignore_index=False)
                print(f"Informações adicionadas com sucesso para o filme: {row['Name']}\n")
            else:
                print(f"\n ******* Título não corresponde ou não encontrado: {row['Name']} != {data.get('Title', 'N/A')} ******* \n")
        except Exception as e:
            print(f"Erro no índice {i}: {e}")
            print("Traceback:")
            traceback.print_exc()


Processando filme 109: Kill Bill: Vol. 2
Informações adicionadas com sucesso para o filme: Kill Bill: Vol. 2
Processando filme 110: Captain America: Brave New World
Informações adicionadas com sucesso para o filme: Captain America: Brave New World
Processando filme 111: Mickey 17
Informações adicionadas com sucesso para o filme: Mickey 17
Processando filme 112: Longlegs
Informações adicionadas com sucesso para o filme: Longlegs
Processando filme 113: A Dog's Will
Informações adicionadas com sucesso para o filme: A Dog's Will
Processando filme 114: The Monkey
Informações adicionadas com sucesso para o filme: The Monkey
Processando filme 115: A Dog's Will 2
Informações adicionadas com sucesso para o filme: A Dog's Will 2
Processando filme 116: The Accountant
Informações adicionadas com sucesso para o filme: The Accountant
Processando filme 117: Oeste Outra Vez
Informações adicionadas com sucesso para o filme: Oeste Outra Vez


Cleaning unnecessary columns

In [17]:
newest_df.drop(['Date', 'Rewatch', "Tags"], axis='columns', inplace=True)

Updating the new info to newer_diary and open it no make the analysis

In [18]:
newest_df.to_csv(updated_letterboxd_data, index=False)    #*******************************
newest_df = pd.read_csv(updated_letterboxd_data, sep=",")

In [19]:
print(newest_df.shape)

(118, 13)


Watched total time

In [21]:
total_minutes = 0   #*****************************

for i in range(len(newest_df)):
    if pd.notna(newest_df.loc[i, "Runtime"]) and newest_df.loc[i, "Runtime"] != "N/A":
        total_minutes += int(newest_df["Runtime"][i][:3])

print(f"Total watched time: {total_minutes / 60:.2f} hours")


Total watched time: 240.20 hours


Function to separate each topic (Genre, Director, Writer, Actors, Language, Country)

In [21]:
def count_topics(info, topic):    #************************
    if newest_df.loc[i, topic] is not None:
        temp = info[topic].tolist()
        topics = []

        for elem in temp:
            if pd.notna(elem):
                for topic in elem.split(", "):
                    topics.append(topic)

        return Counter(topics)


Function to calculate the mean rating of elements inside a topic

In [22]:
def cal_mean(info, topic):    #****************************
    topic_rating_sum = {}
    topic_rating_count = {}

    # Iterar sobre cada linha do DataFrame
    for i in range(len(info)):
        infos = info.loc[i, topic]
        rating = info.loc[i, "Rating"]

        # Verificar se o valor de 'Genre' não é nulo
        if infos != "N/A" and pd.notna(infos):
            # Separar os gêneros
            info_list = infos.split(", ")
            for p in info_list:
                # Adicionar o rating ao gênero correspondente
                if p not in topic_rating_sum:
                    topic_rating_sum[p] = 0
                    topic_rating_count[p] = 0
                topic_rating_sum[p] += rating
                topic_rating_count[p] += 1

    # Calcular a média de ratings por gênero
    average_rating_by_topic = {
        elem: topic_rating_sum[elem] / topic_rating_count[elem]
        for elem in topic_rating_sum
    }

    return average_rating_by_topic

Function to separate the topic and the most wathed movies, with rating

In [23]:
def separate_topic_and_most_wathed(topic, newest_df):    #************************

    topic_mean = cal_mean(newest_df, topic)

    for i in count_topics(newest_df, topic).most_common(5):
        print(f"{i[0]} => {i[1]} movies, Mean Rating: {topic_mean[i[0]]:.2f}\n")

Function to sorted the dict of each topic (REVERSE=TRUE, to picking up top rating)

In [24]:
def sorted_dict_by_rating(dict_info):       #************************
    return dict(sorted(dict_info.items(), key=lambda item: item[1][1], reverse=True))

Formating data to show the info dict by rating

In [25]:
def format_dict_by_rating(topic, newest_df):     #***********************

    topic_mean = cal_mean(newest_df, topic)
    dict_topic = {}

    for i in count_topics(newest_df, topic).most_common():
        dict_topic[i[0]] = [i[1], topic_mean[i[0]]]

    for item in islice(sorted_dict_by_rating(dict_topic).items(), 5):
        print(f"{topic} > {item[0]} => N WATCHED: {item[1][0]} || MEAN RATING: {item[1][1]:.2f}")

Separate movie ratings by decade

In [26]:
def separate_movie_by_decade(newest_df):         #**********************************
    decades = {}
    min_year = newest_df["Year"].min()
    max_year = newest_df["Year"].max()
    aux_min_year = min_year

    for i in range(min_year, max_year + 1):
        if i % 10 == 0:
            aux_max_year = i
            decades[f"{aux_min_year}-{aux_max_year}"] = None
            aux_min_year = i + 1

    decades[f"{aux_min_year}-{max_year}"] = None

    return decades

Alocate rating per decade

In [27]:
decades = separate_movie_by_decade(newest_df)     #******************************

for key in decades.keys():
    aux_rating = 0
    s = 0

    ini = key[:4]
    end = key[5:]

    for i in range(int(ini) , int(end) + 1):
        year_rating = newest_df[newest_df["Year"] == i]["Rating"].mean()
        if pd.notna(year_rating):
            aux_rating += year_rating
            s += 1

    decades[key] = aux_rating / s if s > 0 else 0

for key in decades:
    print(f"{key} => {decades[key]:.2f} stars")

1976-1980 => 4.00 stars
1981-1990 => 3.94 stars
1991-2000 => 4.34 stars
2001-2010 => 3.68 stars
2011-2020 => 3.59 stars
2021-2025 => 3.07 stars


Total number of watched movies

In [28]:
print(f"{len(newest_df)} movies watched")  #******************************

118 movies watched


Mean value of all ratings

In [29]:
print("Mean rating of {:.2f} stars".format(newest_df["Rating"].mean())) #********************************

Mean rating of 3.46 stars


Year mean of produced movies watched

In [30]:
print("{:.0f}".format(df["Year"].mean()))   #**********************************

2014


Movies logged in a specific year (TOP 5)

In [31]:
from collections import Counter     #********************************
aux = df["Watched Date"].tolist()
w = []

for i in range(len(aux)):
    w.append(aux[i][:4])

w_counter = Counter(w)

for item in w_counter.most_common(len(w_counter)):
    print(f"{item[0]} => {item[1]} watched films")

2024 => 55 watched films
2025 => 37 watched films
2023 => 13 watched films
2021 => 6 watched films
2020 => 4 watched films
2022 => 3 watched films


Year with more watched movies and mean rating (TOP FIVE BY NUMBER OF MOVIES)

In [32]:
from collections import Counter      #*****************************
from itertools import islice

# PEGA O ANO NO DATAFRAME E CONTA QUANTAS VEZES AQUELE ANO APARECEU NA LISTA DE ASSSISTIDOS
years = df["Year"].tolist()
counter = Counter(years)

geral_info = {}

# DICIONÁRIO FORMADO PELA CHAVE (ANO), COM O PRIMEIRO VALOR QUANTIDADE DE FILMES DAQUELE ANO VISTOS E SEGUNDO VALOR MÉDIA DO ANO
for item in counter.most_common(len(counter)):
    geral_info[item[0]] = [item[1], df[(df["Year"] == item[0])]["Rating"].mean().round(2)]

for item in islice(geral_info.items(), 5):  # Limita a 5 itens
    print(f"YEAR > {item[0]} => N WATCHED: {item[1][0]} || MEAN RATING: {item[1][1]}")


YEAR > 2024 => N WATCHED: 23 || MEAN RATING: 3.04
YEAR > 2022 => N WATCHED: 11 || MEAN RATING: 3.05
YEAR > 2023 => N WATCHED: 9 || MEAN RATING: 3.17
YEAR > 2021 => N WATCHED: 6 || MEAN RATING: 3.58
YEAR > 2019 => N WATCHED: 6 || MEAN RATING: 3.92


Year with movies high rated (TOP FIVE BY MEAN RATING)

In [34]:
for item in islice(sorted_dict_by_rating(geral_info).items(), 5):
    print(f"YEAR > {item[0]} => N WATCHED: {item[1][0]} || MEAN RATING: {item[1][1]}")

YEAR > 1998 => N WATCHED: 1 || MEAN RATING: 5.0
YEAR > 1995 => N WATCHED: 1 || MEAN RATING: 5.0
YEAR > 1990 => N WATCHED: 2 || MEAN RATING: 4.5
YEAR > 1992 => N WATCHED: 1 || MEAN RATING: 4.5
YEAR > 2014 => N WATCHED: 3 || MEAN RATING: 4.33


Genres with more wathed movies and the mean Rating (TOP FIVE BY NUMBER OF MOVIES)

In [35]:
separate_topic_and_most_wathed("Genre", newest_df)

Drama => 59 movies, Mean Rating: 3.68

Action => 46 movies, Mean Rating: 3.25

Comedy => 35 movies, Mean Rating: 3.56

Crime => 33 movies, Mean Rating: 3.59

Adventure => 30 movies, Mean Rating: 3.32



Genres with the high mean rating and the number of movies (TOP FIVE BY MEAN RATING)

In [36]:
format_dict_by_rating("Genre", newest_df)

Genre > Biography => N WATCHED: 4 || MEAN RATING: 4.25
Genre > History => N WATCHED: 2 || MEAN RATING: 4.25
Genre > War => N WATCHED: 1 || MEAN RATING: 4.00
Genre > Family => N WATCHED: 1 || MEAN RATING: 4.00
Genre > Sport => N WATCHED: 2 || MEAN RATING: 3.75


Directors with more wathed movies and the mean Rating (TOP FIVE BY NUMBER OF MOVIES)

In [37]:
separate_topic_and_most_wathed("Director", newest_df)

Martin Scorsese => 5 movies, Mean Rating: 4.40

Jordan Peele => 3 movies, Mean Rating: 3.83

Quentin Tarantino => 3 movies, Mean Rating: 4.00

Ruben Fleischer => 2 movies, Mean Rating: 2.50

Adam McKay => 2 movies, Mean Rating: 3.50



Directors with higher rated, and the number of watched movies (TOP FIVE BY RATING)

In [38]:
format_dict_by_rating("Director", newest_df)

Director > Peter Weir => N WATCHED: 1 || MEAN RATING: 5.00
Director > Christopher Nolan => N WATCHED: 2 || MEAN RATING: 4.75
Director > Zack Snyder => N WATCHED: 1 || MEAN RATING: 4.50
Director > Marc Forster => N WATCHED: 1 || MEAN RATING: 4.50
Director > Robert Zemeckis => N WATCHED: 1 || MEAN RATING: 4.50


Writers with more watched movies and the mean Rating (TOP 5 BY NUMBER OF MOVIES)

In [39]:
separate_topic_and_most_wathed("Writer", newest_df)

Kelly Marcel => 3 movies, Mean Rating: 2.67

Jordan Peele => 3 movies, Mean Rating: 3.83

Derek Kolstad => 3 movies, Mean Rating: 3.50

Rhett Reese => 3 movies, Mean Rating: 3.17

Paul Wernick => 3 movies, Mean Rating: 3.17



Writers with higher rated and the number of watched movies (TOP FIVE BY RATING)

In [40]:
format_dict_by_rating("Writer", newest_df)

Writer > Nicholas Pileggi => N WATCHED: 1 || MEAN RATING: 5.00
Writer > Martin Scorsese => N WATCHED: 1 || MEAN RATING: 5.00
Writer > Andrew Niccol => N WATCHED: 1 || MEAN RATING: 5.00
Writer > Andrew Kevin Walker => N WATCHED: 1 || MEAN RATING: 5.00
Writer > Christopher Nolan => N WATCHED: 2 || MEAN RATING: 4.75


Actors with more watched movies and the mean Rating (TOP 5 BY NUMBER OF MOVIES)

In [41]:
separate_topic_and_most_wathed("Actors", newest_df)

Brad Pitt => 5 movies, Mean Rating: 3.90

Tom Hardy => 4 movies, Mean Rating: 2.75

Leonardo DiCaprio => 4 movies, Mean Rating: 4.00

Joe Pesci => 4 movies, Mean Rating: 4.50

Adam Sandler => 3 movies, Mean Rating: 3.50



Actors with higher rated and the number of movies watched (TOP 5 BY RATING)

In [42]:
format_dict_by_rating("Actors", newest_df)

Actors > Ray Liotta => N WATCHED: 1 || MEAN RATING: 5.00
Actors > Ed Harris => N WATCHED: 1 || MEAN RATING: 5.00
Actors > Laura Linney => N WATCHED: 1 || MEAN RATING: 5.00
Actors > Carrie-Anne Moss => N WATCHED: 1 || MEAN RATING: 5.00
Actors > Joe Pantoliano => N WATCHED: 1 || MEAN RATING: 5.00


Movies per language and the mean rating of top 5 (TOP 5 BY NUMBER OF MOVIES)

In [43]:
separate_topic_and_most_wathed("Language", newest_df)

English => 108 movies, Mean Rating: 3.46

Spanish => 35 movies, Mean Rating: 3.47

French => 18 movies, Mean Rating: 3.58

Portuguese => 10 movies, Mean Rating: 3.45

Russian => 10 movies, Mean Rating: 3.35



COUNTRIES WITH HIGHER RATING AND THE NUMBER OF MOVIES WATCHED (TOP 5 BY RATING)

In [44]:
format_dict_by_rating("Country", newest_df)

Country > Sweden => N WATCHED: 2 || MEAN RATING: 4.00
Country > Ireland => N WATCHED: 2 || MEAN RATING: 4.00
Country > United Arab Emirates => N WATCHED: 1 || MEAN RATING: 4.00
Country > Jordan => N WATCHED: 1 || MEAN RATING: 4.00
Country > Gambia => N WATCHED: 1 || MEAN RATING: 4.00


Movies per country and the mean rating of top 5 (TOP 5 BY NUMBER OF MOVIES)

In [45]:
separate_topic_and_most_wathed("Country", newest_df)

United States => 104 movies, Mean Rating: 3.48

Canada => 26 movies, Mean Rating: 3.33

United Kingdom => 22 movies, Mean Rating: 3.50

Japan => 6 movies, Mean Rating: 3.75

France => 6 movies, Mean Rating: 3.58



Languages with higher rated and the number of movies (TOP 5 BY RATING)

In [46]:
format_dict_by_rating("Language", newest_df)

Language > Icelandic => N WATCHED: 1 || MEAN RATING: 4.50
Language > American Sign  => N WATCHED: 1 || MEAN RATING: 4.50
Language > Latvian => N WATCHED: 1 || MEAN RATING: 4.50
Language > Cantonese => N WATCHED: 5 || MEAN RATING: 4.10
Language > Yiddish => N WATCHED: 2 || MEAN RATING: 4.00
